# Binder Test Repo

In [41]:
import pandas as pd
import os 
import numpy as np
import re
import yaml
import urllib
import boto3
import glob
import requests, json

### Configuration for fetching NOAA NOS data

In [50]:
station_id = '8536110'
datum = 'MLLW'
product = 'water_level'
time_zone = 'gmt'
units = 'metric'
file_format = 'csv'
start_dt = '20190101'
end_dt = '20191231'
path = os.path.dirname('/01_fetch/out/')
os.path.isdir(path)
filename = f"noaa_nos_{station_id}_{product}_{start_dt}_{end_dt}.csv"
data_outfile = os.path.join('.', path + '/' + filename)
metadata_path = os.path.dirname('/01_fetch/out/metadata/')
metadata_filename = f"noaa_nos_metadata_{station_id}.csv"
metadata_outfile = os.path.join('.', metadata_path + '/' + metadata_filename)
Bucket = 'drb-estuary-salinity'
session = boto3.Session(profile_name='dev')
s3_client = session.client('s3')
write_location = 'S3'

Fetch Station Data

In [51]:
'''fetch NOAA NOS data from select station. Change product argument for NOAA NOS data product.
(ie. product = current for current data)'''
data_url = f'https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?product=predictions&application=NOS.COOPS.TAC.WL&begin_date=' \
            f'{start_dt}&end_date={end_dt}&datum={datum}&station={station_id}&product={product}&time_zone=' \
            f'{time_zone}&units={units}&interval=&format={file_format}'
urllib.request.urlretrieve(data_url, data_outfile)
if write_location == 'S3':
    print('uploading to s3')
    s3_client.upload_file(data_outfile, bucket, '01_fetch/out/'+os.path.basename(data_outfile))

uploading to s3


Fetch metadata

In [52]:
'''fetch tides and currents metadata from NOAA NOS station'''
metadata_url = f'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/{station_id}/.json'
response = requests.get(metadata_url)
text = response.text
data = json.loads(text)
nested_list = pd.json_normalize(data, record_path=['stations'])
df = nested_list.stack().reset_index()
metadata = df.iloc[1:, 1:].rename(columns={df.columns[1]: "data_property",
                                            df.columns[2]: (station_id)}).replace(".self", "", regex=True)
metadata.to_csv(metadata_outfile, index=False)
if write_location == 'S3':
    print('uploading to s3')
    s3_client.upload_file(metadata_outfile, bucket, '01_fetch/out/'+os.path.basename(metadata_outfile))

uploading to s3


combine csv file

In [ ]:
file_location = 'S3'

In [ ]:
# directory to files
file_dir = os.path.join(f"{file_location}", f"noaa_nos_metadata_*.csv")
# A list of all csv
file_list = glob.glob(file_dir)
# join csvs
df = pd.concat(map(pd.read_csv, file_list), ignore_index=True, sort = True)
if save_location = 'S3"
    #define key to point to datasets for specified station
    key = f'01_fetch/out/noaa_nos_{station_id}'
    #create a list of csv files in s3 bucket with observational data specific to station
    prefix_df = []
    prefix_obs = [obj['Key'] for obj in s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=f'01_fetch/out/noaa_nos_{station_id}')['Contents']]
    #grab each dataset from bucket and combine to a single csv file
        for obs in prefix_obs:
            obj = s3_client.get_object(Bucket=s3_bucket, Key=obs)
            temp = pd.read_csv(obj.get('Body'), index_col=None, header=0, encoding='utf8')
            prefix_df.append(temp)
    #sort columns to timestep
    df = pd.concat(prefix_df, ignore_index=True, sort = True)
    #save to csv file
    if write_location == 'S3':
           print('uploading to s3')
           s3_client.upload_file(data_outfile_csv, s3_bucket, '02_munge/out/'+os.path.basename(data_outfile_csv))